# Save predictions to Pandas

This notebook loads all 2x5 trained models, the two datasets (fast&slow), gets the predictions and saves the results to a file

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [3]:
import os
# do we have the correct env loaded?
assert 'donkey2' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [7]:
# fast
# data
# faster-autodrive-18_20, fastest-autodrive-18_40
# models
# linear
# existing model on full data
# linear-fastest-daytime-full-data-multiply.tflite
# 3D
# existing model on full data
# 3d-fastest-daytime-full-data-multiply.h5
#
# slow
# data
# daytime-manual-slower-1, daytime-manual-slower-2
# models
# linear
# existing model on full data
# linear-steering-slower-daytime-full-data-multiply.tflite
# 3D
# existing model on full data
# 3d-steering-slower-daytime-full-data-multiply.h5
# data is in https://drive.google.com/drive/folders/1CR-QXkKg1k5OFQkmXskcOf_2nUEhkVTC


In [8]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/


### Common settings for both directions

In [10]:
from dataloader import load_data
import donkeycar as dk
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False

DIRECTION = 'CC'
N_FOLDS = 5
DATA_PATH = 'data'


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py



### Load Counter Clockwise tubs and get predictions

In [27]:
from dataloader import load_records

fast_speed_data = load_records('faster-autodrive-18_20', cfg, DATA_PATH, TUBRECORD_SIZE)
fast_speed_data2 = load_records('fastest-autodrive-18_40', cfg, DATA_PATH, TUBRECORD_SIZE)
fast_speed_data.extend(fast_speed_data2)
len(fast_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/faster-autodrive-18_20']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive-18_20/catalog_14.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/fastest-autodrive-18_40']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/fastest-autodrive-18_40/catalog_10.catalog


20771

In [28]:

slow_speed_data = load_records('daytime/daytime-manual-slower-1', cfg, DATA_PATH, TUBRECORD_SIZE)
slow_speed_data2 = load_records('daytime/daytime-manual-slower-2', cfg, DATA_PATH, TUBRECORD_SIZE)
slow_speed_data.extend(slow_speed_data2)
len(slow_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/daytime/daytime-manual-slower-1']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/daytime/daytime-manual-slower-1/catalog_10.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/daytime/daytime-manual-slower-2']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/daytime/daytime-manual-slower-2/catalog_21.catalog


31361

In [29]:
from donkeycar.utils import get_model_by_type

model_type = '3d-modified-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d_42 (Conv3D)          (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d_28 (MaxPoolin  (None, 1, 10, 26, 16)    0         
 g3D)                                                            
                                                                 
 conv3d_43 (Conv3D)          (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_29 (MaxPoolin  (None, 1, 5, 13, 32)     0         
 g3D)                                                            
                                                                 
 conv3d_44 (Conv3D)          (None, 1, 5, 13, 32)      9248  

In [30]:
from losses import get_ground_truth
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


seq size 3
len ground_truth 20771
seq size 3
len ground_truth 31361


In [42]:
from losses import get_predictions
from model_loader import load_model
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=N_FOLDS, shuffle=False)


train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d'

for fold, (train_index, test_index) in enumerate(kf.split(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


130/130 [==============================] - 149s 1s/step
predictions shape (16616,)
33/33 [==============================] - 42s 1s/step
predictions shape (4155,)
246/246 [==============================] - 279s 1s/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (31361,)
130/130 [==============================] - 29s 219ms/step
predictions shape (16617,)
33/33 [==============================] - 7s 220ms/step
predictions shape (4154,)
246/246 [==============================] - 52s 210ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


predictions shape (31361,)
130/130 [==============================] - 27s 209ms/step
predictions shape (16617,)
33/33 [==============================] - 7s 213ms/step
predictions shape (4154,)
246/246 [==============================] - 53s 214ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


predictions shape (31361,)
130/130 [==============================] - 28s 218ms/step
predictions shape (16617,)
33/33 [==============================] - 7s 208ms/step
predictions shape (4154,)
246/246 [==============================] - 51s 209ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


predictions shape (31361,)
130/130 [==============================] - 27s 211ms/step
predictions shape (16617,)
33/33 [==============================] - 7s 217ms/step
predictions shape (4154,)
246/246 [==============================] - 51s 207ms/step
predictions shape (31361,)


In [43]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.24826560909003662, -0.24429040840793703, -0.26214615512547934)

In [45]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((20771,), (20771,), (31361,))

In [71]:
np.savetxt('open-loop-results/daytime-fast-3d-model-daytime-fast-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.savetxt('open-loop-results/daytime-fast-3d-model-daytime-slow-data-prediction-and-ground-truth.csv.gz', np.array([slow_predictions, slow_ground_truths]).T)

In [73]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d'

for fold, (train_index, test_index) in enumerate(kf.split(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


196/196 [==============================] - 54s 274ms/step
predictions shape (25088,)
50/50 [==============================] - 14s 271ms/step
predictions shape (6273,)
163/163 [==============================] - 44s 270ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (20771,)
197/197 [==============================] - 53s 271ms/step
predictions shape (25089,)
49/49 [==============================] - 14s 275ms/step
predictions shape (6272,)
163/163 [==============================] - 47s 287ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


predictions shape (20771,)
197/197 [==============================] - 54s 272ms/step
predictions shape (25089,)
49/49 [==============================] - 18s 367ms/step
predictions shape (6272,)
163/163 [==============================] - 53s 321ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


predictions shape (20771,)
197/197 [==============================] - 54s 272ms/step
predictions shape (25089,)
49/49 [==============================] - 15s 297ms/step
predictions shape (6272,)
163/163 [==============================] - 47s 290ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-3d/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


predictions shape (20771,)
197/197 [==============================] - 56s 282ms/step
predictions shape (25089,)
49/49 [==============================] - 14s 295ms/step
predictions shape (6272,)
163/163 [==============================] - 47s 288ms/step
predictions shape (20771,)


In [74]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.1984441325308632, -0.1997503545546858, -0.1432111012440308)

In [75]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((31361,), (31361,), (20771,))

In [76]:
np.savetxt('open-loop-results/daytime-slow-3d-model-daytime-slow-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, slow_ground_truths]).T)

np.savetxt('open-loop-results/daytime-slow-3d-model-daytime-fast-data-prediction-and-ground-truth.csv.gz', np.array([ fast_predictions, fast_ground_truths]).T)


In [79]:
cfg.SEQUENCE_LENGTH = 0
cfg.ROI_CROP_TOP = 59


In [80]:
model_type = 'linear-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter


Model: "linear"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_in (InputLayer)            [(None, 61, 160, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 29, 78, 24)   1824        ['img_in[0][0]']                 
                                                                                                  
 dropout_108 (Dropout)          (None, 29, 78, 24)   0           ['conv2d_1[0][0]']               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 13, 37, 32)   19232       ['dropout_108[0][0]']       

In [84]:
fast_speed_data = load_records('faster-autodrive-18_20', cfg, DATA_PATH, TUBRECORD_SIZE)
fast_speed_data2 = load_records('fastest-autodrive-18_40', cfg, DATA_PATH, TUBRECORD_SIZE)
fast_speed_data.extend(fast_speed_data2)
len(fast_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/faster-autodrive-18_20']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive-18_20/catalog_14.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/fastest-autodrive-18_40']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/fastest-autodrive-18_40/catalog_10.catalog


20797

In [85]:

slow_speed_data = load_records('daytime/daytime-manual-slower-1', cfg, DATA_PATH, TUBRECORD_SIZE)
slow_speed_data2 = load_records('daytime/daytime-manual-slower-2', cfg, DATA_PATH, TUBRECORD_SIZE)
slow_speed_data.extend(slow_speed_data2)
len(slow_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/daytime/daytime-manual-slower-1']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/daytime/daytime-manual-slower-1/catalog_10.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/daytime/daytime-manual-slower-2']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/daytime/daytime-manual-slower-2/catalog_21.catalog


31379

In [91]:
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


len ground_truth 20797
len ground_truth 31379


In [100]:

train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear'

for fold, (train_index, test_index) in enumerate(kf.split(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions[:,0]

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5


130/130 [==============================] - 17s 131ms/step
predictions shape (16637, 1)
33/33 [==============================] - 4s 123ms/step
predictions shape (4160, 1)
246/246 [==============================] - 30s 122ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering


predictions shape (31379, 1)


INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5


...

predictions shape (31379, 1)
130/130 [==============================] - 14s 105ms/step
predictions shape (16638, 1)
33/33 [==============================] - 3s 104ms/step
predictions shape (4159, 1)
246/246 [==============================] - 26s 104ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5


predictions shape (31379, 1)
130/130 [==============================] - 14s 105ms/step
predictions shape (16638, 1)
33/33 [==============================] - 3s 103ms/step
predictions shape (4159, 1)
246/246 [==============================] - 26s 104ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-fast-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5


predictions shape (31379, 1)
130/130 [==============================] - 13s 103ms/step
predictions shape (16638, 1)
33/33 [==============================] - 3s 103ms/step
predictions shape (4159, 1)
246/246 [==============================] - 25s 101ms/step
predictions shape (31379, 1)


In [101]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.2582955911659102, -0.2580214005839385, -0.2849443084188833)

In [102]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((20797,), (20797,), (31379,))

In [103]:
np.savetxt('open-loop-results/daytime-fast-linear-model-daytime-fast-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.savetxt('open-loop-results/daytime-fast-linear-model-daytime-slow-data-prediction-and-ground-truth.csv.gz', np.array([slow_predictions, slow_ground_truths]).T)

In [104]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear'

for fold, (train_index, test_index) in enumerate(kf.split(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions[:,0]

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5


197/197 [==============================] - 20s 100ms/step
predictions shape (25103, 1)
50/50 [==============================] - 5s 102ms/step
predictions shape (6276, 1)
163/163 [==============================] - 16s 100ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5


predictions shape (20797, 1)
197/197 [==============================] - 20s 100ms/step
predictions shape (25103, 1)
50/50 [==============================] - 5s 101ms/step
predictions shape (6276, 1)
163/163 [==============================] - 16s 100ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering


...

197/197 [==============================] - 20s 101ms/step
predictions shape (25103, 1)
50/50 [==============================] - 5s 103ms/step
predictions shape (6276, 1)
163/163 [==============================] - 16s 101ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5


predictions shape (20797, 1)
197/197 [==============================] - 20s 101ms/step
predictions shape (25103, 1)
50/50 [==============================] - 5s 102ms/step
predictions shape (6276, 1)
163/163 [==============================] - 17s 102ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/daytime-slow-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5


predictions shape (20797, 1)
197/197 [==============================] - 20s 102ms/step
predictions shape (25104, 1)
50/50 [==============================] - 5s 101ms/step
predictions shape (6275, 1)
163/163 [==============================] - 16s 100ms/step
predictions shape (20797, 1)


In [105]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.23733802340388604, -0.236945208646325, -0.19236103558606835)

In [106]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((31379,), (31379,), (20797,))

In [107]:
np.savetxt('open-loop-results/daytime-slow-linear-model-daytime-slow-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, slow_ground_truths]).T)

np.savetxt('open-loop-results/daytime-slow-linear-model-daytime-fast-data-prediction-and-ground-truth.csv.gz', np.array([ fast_predictions, fast_ground_truths]).T)